In [1]:
# Import all the necessary libraries
import pygame
import time
from pygame.locals import *
import random

pygame 2.5.2 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# parameters
SIZE = 40
DISPLAY_SIZE = (25*SIZE, 20*SIZE)
FONT_COLOR = (255,255,255)

In [3]:
# defining APPLE class
class Apple:
    def __init__(self, parent_screen):
        self.parent_screen = parent_screen
        self.image = pygame.image.load("apple.jpg").convert()
        self.x = SIZE*4
        self.y = SIZE*4

    # drawing the apple
    def draw(self):
        self.parent_screen.blit(self.image, (self.x, self.y))
        pygame.display.flip()

    # moving the apple to a random location
    def move(self):
        self.x = random.randint(1,24)*SIZE
        self.y = random.randint(1,19)*SIZE

In [4]:
# defining SNAKE class
class Snake:
    def __init__(self, surface):
        self.parent_screen = surface
        self.image = pygame.image.load("block.jpg").convert()
        self.direction = 'right'  # initial direction
        self.length = 1    # initial length
        self.x = [40]
        self.y = [40]

    # different motions using arrow keys
    def move_left(self):
        self.direction = 'left'

    def move_right(self):
        self.direction = 'right'

    def move_up(self):
        self.direction = 'up'

    def move_down(self):
        self.direction = 'down'

    # movement of snake
    def walk(self):
        for i in range(self.length-1,0,-1):
            self.x[i] = self.x[i-1]
            self.y[i] = self.y[i-1]

        if self.direction == 'left':
            self.x[0] -= SIZE
        if self.direction == 'right':
            self.x[0] += SIZE
        if self.direction == 'up':
            self.y[0] -= SIZE
        if self.direction == 'down':
            self.y[0] += SIZE
        self.draw()

    # drawing the snake
    def draw(self):
        for i in range(self.length):
            self.parent_screen.blit(self.image, (self.x[i], self.y[i]))
        pygame.display.flip()

    # increasing the length of snake
    def increase_length(self):
        self.length += 1
        self.x.append(-1)
        self.y.append(-1)

In [5]:
# defining GAME class
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Snake Game by Parth")
        self.surface = pygame.display.set_mode(DISPLAY_SIZE)
        self.snake = Snake(self.surface)
        self.snake.draw()
        self.apple = Apple(self.surface)
        self.apple.draw()

    # reset the snake and apple
    def reset(self):
        self.snake = Snake(self.surface)
        self.apple = Apple(self.surface)

    # condition for collision
    def is_collision(self, x1, y1, x2, y2):
        if x1 >= x2 and x1 < x2 + SIZE:
            if y1 >= y2 and y1 < y2 + SIZE:
                return True
        return False
    
    # background image
    def render_background(self):
        bg = pygame.image.load("background.jpg")
        self.surface.blit(bg, (0,0))

    # display score
    def display_score(self):
        font = pygame.font.SysFont('arial',30)
        score = font.render(f"Score: {self.snake.length}",True,FONT_COLOR)
        self.surface.blit(score,(850,10))

    # play the game
    def play(self):
        self.render_background()
        self.snake.walk()
        self.apple.draw()
        self.display_score()
        pygame.display.flip()

        # snake eating apple scenario
        if self.is_collision(self.snake.x[0], self.snake.y[0],self.apple.x,self.apple.y):
            self.snake.increase_length()
            self.apple.move()

        # snake colliding with itself
        for i in range(2,self.snake.length):
            if self.is_collision(self.snake.x[0], self.snake.y[0],self.snake.x[i],self.snake.y[i]):
                raise "Collision occured"
            
        # snake colliding with the wall
        if not (0<= self.snake.x[0] < DISPLAY_SIZE[0] and 0<= self.snake.y[0] < DISPLAY_SIZE[1]):
            raise "Hit the boundry error"

    # show game over 
    def show_game_over(self):
        self.render_background()
        font = pygame.font.SysFont('arial', 30)
        line1 = font.render(f"Game is over! Your score is {self.snake.length}", True, FONT_COLOR)
        self.surface.blit(line1, (200, 300))
        line2 = font.render("To play again press Enter. To exit press Escape!", True, FONT_COLOR)
        self.surface.blit(line2, (200, 350))
        pygame.display.flip()

    # run the game
    def run(self):
        running = True
        pause = False

        while running:
            for event in pygame.event.get():
                if event.type == KEYDOWN:
                    if event.key == K_ESCAPE:
                        running = False

                    if event.key == K_RETURN:
                        pause = False

                    if not pause:
                        if event.key == K_LEFT:
                            self.snake.move_left()

                        if event.key == K_RIGHT:
                            self.snake.move_right()

                        if event.key == K_UP:
                            self.snake.move_up()

                        if event.key == K_DOWN:
                            self.snake.move_down()

                elif event.type == QUIT:
                    running = False
            
            try:
                if not pause:
                    self.play()

            except Exception as e:
                self.show_game_over()
                pause = True
                self.reset()

            time.sleep(max(0.1, 0.25 - 0.01*self.snake.length))  # increase the speed with score

In [6]:
# main function
game = Game()
game.run()